In [ ]:
## NEED HOURLY OR MINUTE BASED DATA. DAILY CLOSE IS NOT SUFFICIENT






In [112]:
import requests
import json
import pandas as pd
import os
import numpy as np
import hvplot.pandas
from utils.utils import coin_gecko_cleaner
%matplotlib inline

In [113]:
# Pulling in api data for USDC
DEI_url = 'https://api.coingecko.com/api/v3/coins/dei-token/market_chart/range?vs_currency=usd&from=1649550415&to=1654820815'

In [114]:
# Converting api to json format
DEI = requests.get(DEI_url).json()

In [115]:
# Converting json into DataFrame
DEI_df=pd.DataFrame(DEI)

In [116]:
DEI_df.head()

,prices,market_caps,total_volumes
0,"[1649553547765, 0.9955887381663276]","[1649553547765, 91053460.88962154]","[1649553547765, 2778620.1513326904]"
1,"[1649557135469, 0.99610226529924]","[1649557135469, 91196182.9092828]","[1649557135469, 2702064.674406079]"
2,"[1649560714524, 0.9962916084609464]","[1649560714524, 90852683.03472342]","[1649560714524, 2492731.3445999757]"
3,"[1649564322210, 0.9954580155660735]","[1649564322210, 90868319.17995167]","[1649564322210, 2476517.2885368485]"
4,"[1649567937589, 0.995457181070162]","[1649567937589, 90945502.61133009]","[1649567937589, 2383113.4709684164]"


In [117]:
# The data is returned with epoch time listed as part of the array for every list item. We want to set our index as epoch time, and then change that to DateTime for clarity. 
# The Epoch time also had extra zeros included at the end, which skewed it when converted to regular Datetime. 
# We will address those issues with the following functions:
def extract_epoch_time(item):
    # item is expected to be an array where the 1st element in the array is the epoch time. 
    # this function returns the epoch time.
    return item[0]

def extract_value(item):
    # item is expected to be an array where the 2nd element in the array is the value. 
    # this function returns the value.
    return item[1]

In [118]:
# cleaning url
coin_gecko_cleaner(DEI_url)

,prices,market_caps,total_volumes
time,,,
2022-04-09,0.995589,9.105346e+07,2.778620e+06
2022-04-10,0.995458,9.086832e+07,2.476517e+06
2022-04-11,0.996308,9.122021e+07,2.675049e+06
2022-04-12,0.994491,8.937345e+07,2.926615e+06
2022-04-13,0.996244,8.949043e+07,3.468121e+06
...,...,...,...
2022-06-05,0.545728,3.277484e+07,4.481435e+05
2022-06-06,0.578497,3.474365e+07,3.617336e+05
2022-06-07,0.598043,3.581991e+07,6.359669e+05


In [119]:
DEI = coin_gecko_cleaner(DEI_url)
display(DEI)
type(DEI['prices'])

,prices,market_caps,total_volumes
time,,,
2022-04-09,0.995589,9.105346e+07,2.778620e+06
2022-04-10,0.995458,9.086832e+07,2.476517e+06
2022-04-11,0.996308,9.122021e+07,2.675049e+06
2022-04-12,0.994491,8.937345e+07,2.926615e+06
2022-04-13,0.996244,8.949043e+07,3.468121e+06
...,...,...,...
2022-06-05,0.545728,3.277484e+07,4.481435e+05
2022-06-06,0.578497,3.474365e+07,3.617336e+05
2022-06-07,0.598043,3.581991e+07,6.359669e+05


pandas.core.series.Series

In [132]:
## Define Desired Deviation

deviation_high = 1.01
deviation_low = .99

In [133]:
## List Initiation
current_price = DEI['prices'].iloc[0]
print(current_price)
type(current_price)

0.9955887381663276


numpy.float64

In [142]:
trigger = 0 


for price_to_check in DEI['prices']:
    if price_to_check < (current_price * deviation_low):
        current_price=price_to_check
        trigger= trigger+1
        print(trigger)
        
    if price_to_check > (current_price * deviation_high):
        current_price=price_to_check
        trigger=trigger+1
        print(trigger)
    

    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [143]:
trigger_counter = 0
for price_to_check in DEI['prices']:
    max_deviation_low = deviation_low * current_price 
    max_deviation_high = deviation_high * current_price 
    
    if price_to_check < max_deviation_low:
        print("Found a LOW deviation. Price update required")
        print("Price being compared: " + str(price_to_check))
        print("Max low deviation: " + str(max_deviation_low))
        print("Setting new \"current price\" to: " + str(price_to_check))
        current_price = price_to_check # take the price we just looked at and set it as the new baseline for deviation calculations
        trigger_counter = trigger_counter + 1 # add one to our count of times triggered
        print("Times trigger has been... triggered: " + str(trigger_counter) + "\n")
        
    if price_to_check > max_deviation_high:
        print("Found a HIGH deviation. Price update required")
        print("Price being compared: " + str(price_to_check))
        print("Max high deviation: " + str(max_deviation_high))
        print("Setting new \"current price\" to: " + str(price_to_check))
        current_price = price_to_check # take the price we just looked at and set it as the new baseline for deviation calculations
        trigger_counter = trigger_counter + 1 # add one to our count of times triggered 
        print("Times trigger has been... triggered: " + str(trigger_counter) + "\n")

        

Found a HIGH deviation. Price update required
Price being compared: 0.9955887381663276
Max high deviation: 0.7542182067720666
Setting new "current price" to: 0.9955887381663276
Times trigger has been... triggered: 1

Found a LOW deviation. Price update required
Price being compared: 0.9821553412039963
Max low deviation: 0.9856328507846643
Setting new "current price" to: 0.9821553412039963
Times trigger has been... triggered: 2

Found a HIGH deviation. Price update required
Price being compared: 0.9963837127069648
Max high deviation: 0.9919768946160362
Setting new "current price" to: 0.9963837127069648
Times trigger has been... triggered: 3

Found a LOW deviation. Price update required
Price being compared: 0.9863701317043754
Max low deviation: 0.9864198755798952
Setting new "current price" to: 0.9863701317043754
Times trigger has been... triggered: 4

Found a LOW deviation. Price update required
Price being compared: 0.8431205984232909
Max low deviation: 0.9765064303873316
Setting new 